In [ ]:
import datetime as dt
import requests
import pandas as pd
import plotly.graph_objects as go
import streamlit as st

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperamos variables del contexto de app.py
palette = locals().get("active_palette", ["#0e1624", "#889064"])
active_font = locals().get("active_font", "sans-serif")

# Asignación de colores
COLOR_TOTAL = palette[0]
COLOR_PC = palette[1] if len(palette) > 1 else palette[0]

# Configuración APIs
COUNTRY_ID = "MEX"
INDICATORS = {
    "total": "NGDPD",     # GDP, current prices — Billions of U.S. dollars
    "pc":    "NGDPDPC",   # GDP per capita, current prices — U.S. dollars per capita
}
YEARS_AHEAD = 5
BANXICO_TOKEN = "7c7245244cd2df18b2b03e0834258450b2ab7c578910115fb8975a7f1c48b9e8"
BASE_IMF = "https://www.imf.org/external/datamapper/api/v1"
BASE_BMX = "https://www.banxico.org.mx/SieAPIRest/service/v1"

# -----------------------------------------------------------------------------
# 2. FUNCIONES DE DATOS (CON CACHÉ)
# -----------------------------------------------------------------------------

@st.cache_data(show_spinner=False)
def get_imf_series(indicator_id: str, country_id: str) -> pd.Series:
    """Obtiene series del FMI."""
    url = f"{BASE_IMF}/{indicator_id}/{country_id}"
    try:
        r = requests.get(url, timeout=20)
        r.raise_for_status()
        payload = r.json()

        # Navegar JSON FMI
        node = payload.get("values", {}).get(indicator_id, {}).get(country_id, {})
        if not node:
            return pd.Series(dtype=float)

        # Convertir a serie ordenada
        s = pd.Series({int(y): float(v) for y, v in node.items() if v is not None}).sort_index()
        return s
    except Exception as e:
        st.error(f"Error conectando con FMI ({indicator_id}): {e}")
        return pd.Series(dtype=float)

@st.cache_data(show_spinner=False)
def get_banxico_fix_oportuno(token: str):
    """Obtiene el TC FIX oportuno de Banxico."""
    url = f"{BASE_BMX}/series/SF43718/datos/oportuno"
    try:
        r = requests.get(url, params={"token": token}, timeout=10)
        r.raise_for_status()
        js = r.json()
        serie = js["bmx"]["series"][0]
        dato = float(serie["datos"][0]["dato"])
        fecha = serie["datos"][0]["fecha"]
        return dato, fecha
    except Exception as e:
        # Fallback silencioso pero seguro
        return 20.0, "Estimado"

def select_projection_years(all_years, start_year, n):
    future = [y for y in all_years if y >= start_year]
    return future[:n]

# -----------------------------------------------------------------------------
# 3. GENERACIÓN DE GRÁFICAS
# -----------------------------------------------------------------------------

def plot_projection(df, x_col, y_col, title, color_bar, format_prefix, tooltip_prefix):
    fig = go.Figure()

    # Barra Horizontal
    fig.add_trace(go.Bar(
        y=df[x_col].astype(str), # Años en el eje Y
        x=df[y_col],             # Valor en el eje X
        orientation='h',
        marker_color=color_bar,
        name=title,
        # Custom data para tooltip (Muestra USD original)
        customdata=df["Valor_USD"],
        hovertemplate=(
            "<b>Año %{y}</b><br>" +
            f"MXN: {format_prefix}%{{x:,.0f}}<br>" +
            f"USD: {tooltip_prefix}%{{customdata:,.1f}}<extra></extra>"
        )
    ))

    fig.update_layout(
        title=dict(
            text=title,
            x=0.5,              # <--- TÍTULO CENTRADO
            xanchor='center',
            font=dict(size=18)
        ),
        font=dict(family=active_font),
        template="plotly_white",
        separators=".,",
        height=400,
        yaxis=dict(
            autorange="reversed", # Años recientes arriba
            type='category',
            title=""
        ),
        xaxis=dict(
            showgrid=True,
            gridcolor='#eee',
            title="",
            zeroline=False
        ),
        margin=dict(l=20, r=20, t=50, b=20)
    )

    st.plotly_chart(fig, use_container_width=True)


# -----------------------------------------------------------------------------
# 4. EJECUCIÓN PRINCIPAL
# -----------------------------------------------------------------------------

st.markdown("### 🔮 Proyección del PIB (FMI WEO)")

# Spinner unificado para todas las cargas
with st.spinner("Consultando proyecciones del FMI y Tipo de Cambio actual..."):
    # 1. Obtener datos crudos
    s_total_usd = get_imf_series(INDICATORS["total"], COUNTRY_ID)
    s_pc_usd = get_imf_series(INDICATORS["pc"], COUNTRY_ID)

    # 2. Obtener TC
    fx_fix, fx_date = get_banxico_fix_oportuno(BANXICO_TOKEN)

if not s_total_usd.empty and not s_pc_usd.empty:

    # 3. Filtrar años
    start_year = dt.date.today().year
    target_years = select_projection_years(list(s_total_usd.index), start_year, YEARS_AHEAD)

    if target_years:
        # Preparar DataFrames
        # TOTAL
        df_total = pd.DataFrame({
            "Year": target_years,
            "Valor_USD": s_total_usd.reindex(target_years).values
        })
        # Conversión a MXN (El dato del FMI NGDPD ya viene en BILLIONS, al multiplicar por pesos
        # obtenemos "Billones de Pesos")
        df_total["Valor_MXN"] = df_total["Valor_USD"] * fx_fix

        # PER CAPITA
        df_pc = pd.DataFrame({
            "Year": target_years,
            "Valor_USD": s_pc_usd.reindex(target_years).values
        })
        # Conversión a MXN (Unidades simples)
        df_pc["Valor_MXN"] = df_pc["Valor_USD"] * fx_fix

        # --- UI DE RESULTADOS ---
        st.info(f"Proyecciones calculadas usando Tipo de Cambio FIX: **${fx_fix:.2f} MXN/USD** (Fecha: {fx_date})")

        tab1, tab2 = st.tabs(["💰 PIB Total (Billones MXN)", "👤 PIB Per Cápita (MXN)"])

        with tab1:
            plot_projection(
                df_total,
                "Year",
                "Valor_MXN",
                "Proyección PIB Nacional Total (Billones MXN)",
                COLOR_TOTAL,
                "$",
                "$"
            )
            
            # --- TABLA DE DATOS (PIB TOTAL) ---
            st.markdown("**Datos detallados (Proyección PIB Total)**")
            
            # Preparamos copia para visualización
            df_disp_total = df_total.copy()
            df_disp_total = df_disp_total.rename(columns={
                "Year": "Año",
                "Valor_MXN": "MXN (Billones)",
                "Valor_USD": "USD (Billones)"
            })
            
            st.dataframe(
                df_disp_total,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Año": st.column_config.TextColumn("Año"),
                    "MXN (Billones)": st.column_config.NumberColumn(format="$%.2f B"),
                    "USD (Billones)": st.column_config.NumberColumn(format="$%.2f B")
                }
            )

        with tab2:
            plot_projection(
                df_pc,
                "Year",
                "Valor_MXN",
                "Proyección PIB Per Cápita (Pesos)",
                COLOR_PC,
                "$",
                "$"
            )
            
            # --- TABLA DE DATOS (PIB PER CÁPITA) ---
            st.markdown("**Datos detallados (Proyección Per Cápita)**")
            
            # Preparamos copia para visualización
            df_disp_pc = df_pc.copy()
            df_disp_pc = df_disp_pc.rename(columns={
                "Year": "Año",
                "Valor_MXN": "MXN (Pesos)",
                "Valor_USD": "USD (Dólares)"
            })
            
            st.dataframe(
                df_disp_pc,
                use_container_width=True,
                hide_index=True,
                column_config={
                    "Año": st.column_config.TextColumn("Año"),
                    "MXN (Pesos)": st.column_config.NumberColumn(format="$%.2f"),
                    "USD (Dólares)": st.column_config.NumberColumn(format="$%.2f")
                }
            )

        st.caption("Fuente: Fondo Monetario Internacional (WEO) y Banco de México.")

    else:
        st.warning("El FMI aún no publica proyecciones para los años solicitados.")
else:
    st.error("No se pudieron obtener los datos del FMI. Verifica la conexión.")

Usando Tipo de Cambio: 18.291 MXN/USD (02/12/2025)


Gráfica generada: pib_mex_proyeccion.html
